In [ ]:
import pandas as pd

data_file = 'data/snyk_and_advisory.csv'
df_combined = pd.read_csv(data_file)

df_combined

In [ ]:
import requests
import time 
from tqdm import tqdm  # Import tqdm for the progress bar


filtered_df = df_combined[df_combined["CWE"] != "CWE-506"]
filtered_df = filtered_df[filtered_df['Platform'] == 'npm']

# filtered_df = filtered_df.head(2)


api_key = "your_api_key"
base_url = "https://libraries.io/api"
def fetch_versions(row):
    while True: 
        try:
            platform = row["Platform"].lower()
            package = row["Package"].replace('/', '%2F')
            # Add API Token 
            # api_url =  f"{base_url}/{platform}/{package}?api_key=XXXXX"
            api_url =  f"{base_url}/{platform}/{package}"
            # print(api_url)
            time.sleep(1)
            response = requests.get(api_url)
            if response.status_code == 200:
                data = response.json()
                return data.get("versions", [])
            elif response.status_code == 429:
                print(f"429 Forbidden encountered. Pausing for 1 minute...")
                time.sleep(60)  # Wait for 1 minute if rate limited
            else:
                print(f"Unexpected error {response.status_code}. Retrying...")
                time.sleep(60)  # Small delay before retrying
                break
        except Exception as e:
            print(f"An error occurred: {e}. Retrying...")

tqdm.pandas(desc="Fetching versions")

# Apply the function to fetch versions
filtered_df["Versions"] = filtered_df.progress_apply(lambda row: fetch_versions(row), axis=1)

# Display the final DataFrame
print(filtered_df)

In [ ]:
import requests
import time 
from tqdm import tqdm  # Import tqdm for the progress bar

df_versions = pd.DataFrame()

base_url = "https://libraries.io/api"
def fetch_versions(row):
    while True: 
        try:
            platform_mapping = {
                "Composer": 'packagist',
                # 'pip': 'pipy',
                'Crates': 'cargo'
            }
            platform = platform_mapping.get(row["Platform"], row["Platform"].lower())
            # platform = row["Platform"].lower()
            package = row["Package"].replace('/', '%2F')

            existing_package = df_versions[df_versions['Package'] == package]
            
            if not existing_package.empty and len(str(existing_package['Versions'].iloc[0])) > 10:
                # Package exists, get the versions from the DataFrame
                versions = existing_package['Versions'].iloc[0]  # Get versions from the existing row
                return versions
                # print(f"Package '{package}' found in DataFrame with versions: {versions}")
            else:
                # Add API Token 
                # api_url =  f"{base_url}/{platform}/{package}?api_key=XXXXX"
                api_url =  f"{base_url}/{platform}/{package}"
                # print(api_url)
                time.sleep(1)
                response = requests.get(api_url)
                if response.status_code == 200:
                    data = response.json()
                    return data.get("versions", [])
                elif response.status_code == 429:
                    print(f"429 Forbidden encountered. Pausing for 1 minute...")
                    time.sleep(60)  # Wait for 1 minute if rate limited
                else:
                    print(f"Unexpected error on {package}, {response.status_code}. Breaking...")
                    return response.status_code
                    # time.sleep(60)  # Small delay before retrying
                    # break
        except Exception as e:
            print(f"An error occurred: {e}. Retrying...")


for plt in list(set(df_combined['Platform'])):
    # if plt in ["Go", "Unmanaged (C/C++)", "NuGet", 'Maven', 'Composer', 'C/C++']:
        # continue
    # data_file = 'version_crawls/{}.csv'.format(plt)
    # df_version = pd.read_csv(data_file)
    print('Running for...', plt)
    # filtered_df = df_combined[df_combined["CWE"] != "CWE-506"]
    filtered_df = df_combined[df_combined['Platform'] == plt]

    tqdm.pandas(desc="Fetching versions")

    filtered_df["Versions"] = filtered_df.progress_apply(lambda row: fetch_versions(row), axis=1)

    display(filtered_df)

    output_file = "version_crawls/{}".format(plt+'.csv')
    filtered_df.to_csv(output_file, index=False)
    print(f"Output saved to {output_file}")



# 